In [1]:
import os
import json
import matplotlib.pyplot as plt 
import matplotlib.image as image 
import numpy as np
import pandas as pd
import albumentations as A
import albumentations.pytorch
import cv2
import math

import torch
from pytorch_lightning import LightningModule, Trainer
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchmetrics import Accuracy
from torchvision import transforms
from torchvision.datasets import MNIST

import sys
sys.path.append('../')
from utils.dataset import *
import utils.dataset
# from train import PapsClsModel

In [2]:
# https://stackoverflow.com/questions/68782144/pytorch-can-i-group-batches-by-length

In [3]:
df = pd.read_csv('../lbp_data/test.csv')

In [4]:
df.shape

(5998, 14)

In [5]:
dataset = PapsDataset(df)

(5664, 14)


In [6]:
from utils.sampler_by_group import *
group_ids = create_area_groups(dataset, k=3)

Using [0, 100, 140, 200, 300, 400] as bins for aspect ratio quantization
Count of instances per bin: [ 655 1745 2087  966  211]


In [7]:
if False:
    train_sampler = torch.utils.data.distributed.DistributedSampler(dataset)
    #test_sampler = torch.utils.data.distributed.DistributedSampler(dataset_test)
else:
    train_sampler = torch.utils.data.RandomSampler(dataset)
    #test_sampler = torch.utils.data.SequentialSampler(dataset_test)

In [8]:
#train_sampler = torch.utils.data.SequentialSampler(dataset)
#list(train_sampler)

In [9]:
batch_size=8
train_batch_sampler = GroupedBatchSampler(train_sampler, group_ids, batch_size)
#list(train_batch_sampler)

In [10]:
from utils.collate import collate_fn
data_loader = torch.utils.data.DataLoader(
    dataset, batch_sampler=train_batch_sampler, num_workers=4, collate_fn=collate_fn
)

In [11]:
for epoch in range(2) :
    print('*****************************************')
    for i, targets in enumerate(data_loader) :
        images, labels, areas = targets
        if i < 10 :
            print(images.shape)
            print(labels)
            print(areas)
        else :
            break

*****************************************
torch.Size([8, 3, 221, 204])
tensor([2, 1, 3, 0, 1, 3, 3, 1], dtype=torch.uint8)
tensor([179, 155, 177, 159, 197, 143, 179, 184], dtype=torch.int32)
torch.Size([8, 3, 255, 180])
tensor([0, 3, 1, 0, 0, 3, 3, 3], dtype=torch.uint8)
tensor([154, 171, 161, 186, 159, 145, 159, 168], dtype=torch.int32)
torch.Size([8, 3, 153, 134])
tensor([3, 1, 3, 1, 0, 2, 0, 0], dtype=torch.uint8)
tensor([136, 129, 120, 125, 133, 131, 124, 126], dtype=torch.int32)
torch.Size([8, 3, 309, 280])
tensor([0, 0, 1, 1, 3, 0, 0, 0], dtype=torch.uint8)
tensor([223, 224, 215, 206, 227, 294, 216, 226], dtype=torch.int32)
torch.Size([8, 3, 211, 217])
tensor([0, 0, 3, 3, 0, 1, 1, 0], dtype=torch.uint8)
tensor([166, 193, 159, 152, 199, 149, 172, 165], dtype=torch.int32)
torch.Size([8, 3, 162, 159])
tensor([3, 0, 0, 0, 2, 3, 3, 1], dtype=torch.uint8)
tensor([106, 123, 130, 129, 101, 114, 100, 138], dtype=torch.int32)
torch.Size([8, 3, 131, 154])
tensor([0, 1, 2, 0, 0, 3, 0, 1], dt

In [12]:
data_loader = torch.utils.data.DataLoader(
    dataset, batch_sampler=train_batch_sampler, num_workers=4, collate_fn=collate_fn
)

In [13]:
data_loader = torch.utils.data.DataLoader(
    dataset=dataset,
    # batch_size=self.batch_size,
    # shuffle=True,
    batch_sampler=train_batch_sampler,
    num_workers=4,
    collate_fn=collate_fn,
    pin_memory=True
)

In [14]:
batch_img = []
x = np.random.randint(10,50, 5)
y = np.random.randint(10,50, 5)

for i, j in zip(x, y) :
    print(i, j)
    img = np.random.randn(i,j,3)
    batch_img.append(img)

45 28
29 36
31 30
18 39
20 47


In [15]:
for b in batch_img :
    print(b.shape)

(45, 28, 3)
(29, 36, 3)
(31, 30, 3)
(18, 39, 3)
(20, 47, 3)


In [16]:
max_x = max([ b.shape[0] for b in batch_img])
max_y = max([ b.shape[1] for b in batch_img])
min_x = min([ b.shape[0] for b in batch_img])
min_y = min([ b.shape[1] for b in batch_img])
print(max_x, max_y, min_x, min_y)

45 47 18 28


In [17]:
#image = np.pad(image, ((pl_x,pr_x), (pl_y,pr_y), (0,0)), 'constant', constant_values=0)

In [18]:
for b in batch_img :
    print(b.shape)

(45, 28, 3)
(29, 36, 3)
(31, 30, 3)
(18, 39, 3)
(20, 47, 3)


In [19]:
p_image = ([np.pad(x, ((0, max_x - x.shape[0]), (0, max_y - x.shape[1]), (0,0))) for x in batch_img])
for p in p_image :
    print(p.shape)

(45, 47, 3)
(45, 47, 3)
(45, 47, 3)
(45, 47, 3)
(45, 47, 3)


In [20]:
s_image = np.stack([np.pad(x, ((0, max_x - x.shape[0]), (0, max_y - x.shape[1]), (0,0))) for x in batch_img])
s_image.shape

(5, 45, 47, 3)